## Run the ngai-api gateway

In [ ]:
from ng_ai import ng_ai_api_app as app

app.run(port=9999, host='0.0.0.0')

## Connect to ngai_graphd

- From NebulaGraph **Studio**:

Host: `ngai_graphd`, Port: `9669`

- In container network, with console:

```bash
nebula-console -u user -p password -addr ngai_graphd -port 9669
```

- Or connect from the host(outside the container network):

```bash
nebula-console -u user -p password -addr 127.0.0.1 -port 29669
```

### Call ngai API

It's either a cURL call or a UDF query from ngai-graphd.

#### Query from ngai-graphd

```cypher
-- Prepare the write schema
USE demo_basketballplayer;
CREATE TAG IF NOT EXISTS pagerank(pagerank string);
:sleep 20;
-- Call with ng_ai()
RETURN ng_ai("pagerank", ["follow"], ["degree"], "spark", {space: "demo_basketballplayer", max_iter: 10}, {write_mode: "insert"})
```

Where the parameters are:

- `algo_name`: the name of the algorithm, e.g. `pagerank`
- `edge_types`: the edge types to be used in the algorithm, e.g. `["follow"]`
- `edge_weights`: the edge weights to be used in the algorithm, e.g. `["degree"]`
- `mode`: the mode(engine) to be used in the algorithm, e.g. `spark`, `networkx`
- `algo_context`: the context to be used in the algorithm, e.g. `{space: "basketballplayer", max_iter: 10}`

#### Call with cURL

```bash
curl -X POST \
     -H "Content-Type: application/json" \
     -d '{
          "write_context": {
              "write_mode": "insert"
          },
          "read_context": {
              "edge_types": ["follow"],
              "read_mode": "scan",
              "edge_weights": ["degree"]
          },
          "algo_context": {
              "name": "pagerank",
              "space": "basketballplayer"
          }
     }' \
     http://jupyter:9999/api/v0/spark/pagerank
```

## Info on the ng_ai UDF result

We could get the info from the result like:

```cypher
WITH ngdi("pagerank", ["follow"], ["degree"], "spark",
    {space: "demo_basketballplayer", max_iter: 10},
    {write_mode: "insert"}) AS r
UNWIND [key in keys(r) | {key: key, value: r[key]}] AS row
RETURN row.key, row.value
```

## Check algo result

```cypher
MATCH (n:pagerank)
    WITH n.pagerank.pagerank AS pr, n.player.name AS n
    LIMIT 100
RETURN pr, n ORDER BY pr DESC
```